In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()
import os
import yaml
from crewai import Agent, Task, Crew , LLM
#GROQ_API_KEY=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
# os.environ["GROQ_MODEL_NAME"] = "llama3-8b-8192"
os.environ["GROQ_API_KEY"] = "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"

llm = LLM(
    model="groq/llama3-8b-8192",
    temperature=0.7
)

In [2]:
files = {
    'agents':'config/agents.yaml',
    'tasks':'config/tasks.yaml'
}

configs = { } 
for config_type ,file_path in files.items():
    with open(file_path , 'r') as file:
        configs[config_type] = yaml.safe_load(file)

agents_config = configs['agents']
tasks_config = configs['tasks']
print(agents_config,tasks_config)

{'project_planning_agent': {'role': 'The Ultimate Project Planner\n', 'goal': 'To meticulously break down the {project_type} project into actionable tasks, ensuring no detail is overlooked, and setting precise timelines that align with the {project_objectives}.\n', 'backstory': "As a veteran project manager, youâ€™ve led numerous successful projects, particularly in {industry}. Your keen eye for detail and strategic thinking have always ensured that projects are delivered on time and within scope. Now, you're tasked with planning the next groundbreaking {project_type} project.\n", 'allow_delegation': False, 'verbose': True}, 'estimation_agent': {'role': 'Expert Estimation Analyst\n', 'goal': 'Provide highly accurate time, resource, and effort estimations for each task in the {project_type} project to ensure it is delivered efficiently and on budget.\n', 'backstory': 'You are the go-to expert for project estimation in {industry}. With a wealth of experience and access to vast historical

In [3]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")


In [4]:
project_planning_agent = Agent(
  config=agents_config['project_planning_agent'],
    llm=llm
)

estimation_agent = Agent(
  config=agents_config['estimation_agent'],
    llm=llm
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent'],
    llm=llm
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

In [5]:
from IPython.display import display, Markdown

project = 'Business'
industry = 'Technology'
project_objectives = 'Create a strategy for a small business'
team_members = """
- Jason W (Project Manager)
- John D (Marketing Manager)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



In [6]:
# The given Python dictionary
print(project)
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)
    

Website


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 99ef2e83-5aa2-4f59-9644-1aeb1593c5f0                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: e232ec00-92b0-47cc-b336-38923bb787e9
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: e232ec00-92b0-47cc-b336-38923bb787e9
       Status: Executing Task...
    └── 🤖 Agent: The Ultimate Project Planner
        
            Status: In Progress

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

...

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama3-8b-8192` in organization `org_01jnjfdafne598vqrxtpb8ydwt` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5533, Requested 2429. Please try again in 19.615s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

 An unknown error occurred. Please check the details below.
 Error details: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama3-8b-8192` in organization `org_01jnjfdafne598vqrxtpb8ydwt` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5533, Requested 2429. Please try again in 19.615s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}



🚀 Crew: crew
├── 📋 Task: e232ec00-92b0-47cc-b336-38923bb787e9
│      Assigned to: The Ultimate Project Planner
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: The Ultimate Project Planner
│       
│           Status: ✅ Completed
├── 📋 Task: cd6c4d42-cf25-4ee6-8e52-4088e3b3c0ea
│      Assigned to: Expert Estimation Analyst
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Expert Estimation Analyst
│       
│           Status: ✅ Completed
└── 📋 Task: 9d67a013-5410-47ac-9dc6-9a99da474ae1
       Assigned to: Resource Allocation Strategist
    
       Status: ❌ Failed
    └── 🤖 Agent: Resource Allocation Strategist
        
            Status: In Progress
        └── ❌ LLM Failed

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 9d67a013-5410-47ac-9dc6-9a99da474ae1                                                                     │
│  Agent: Resource Allocation Strategist                                                                          │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 99ef2e83-5aa2-4f59-9644-1aeb1593c5f0                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama3-8b-8192` in organization `org_01jnjfdafne598vqrxtpb8ydwt` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5533, Requested 2429. Please try again in 19.615s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
